In [1]:
import shutil

source_file_path = '.\..\.env'

destination_file_path = '.'

shutil.copy(source_file_path, destination_file_path)

<>:3: SyntaxWarning: invalid escape sequence '\.'
<>:3: SyntaxWarning: invalid escape sequence '\.'
C:\Users\F07\AppData\Local\Temp\ipykernel_10992\2607221228.py:3: SyntaxWarning: invalid escape sequence '\.'
  source_file_path = '.\..\.env'


'.\\.env'

In [2]:
import django
import os
import sys
project_path = '../'  # Adjust this to your actual project path
sys.path.append(project_path)
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'oogway.settings')
django.setup()

In [3]:
from PostAnalyzer.models import (
    Channel,
    PositionSide,
    Market,
    PostStatus,
    MarginMode,
    Symbol,

)
from Shared.Exchange import exchange
from Shared.Constant import PostStatusTypeValues, MarketValues, PositionSideValues, MarginModeValues
from asgiref.sync import sync_to_async
from dotenv import dotenv_values

_config = dotenv_values(".env")



In [4]:
async def add_new_channel(channel_id, name, can_trade):
    # Check if already exists in the database
    existing = await sync_to_async(Channel.objects.filter(name=name, channel_id=channel_id).exists)()

    if not existing:
        # Create a new Channel instance
        new_channel = Channel(
            channel_id=channel_id,
            name=name,
            can_trade=can_trade
        )
        
        # Save the new channel to the database
        await sync_to_async(new_channel.save)()



In [5]:
async def add_post_statuses():
    for status in PostStatusTypeValues:
        for type in status.value[1:]:

            name = status.value[0]
            type_value = int(type)

            # Check if already exists in the database
            existing = await sync_to_async(PostStatus.objects.filter(name=name, type=type_value).exists)()
            
            if not existing:
                # Create if it doesn't exist
                position_side = PostStatus(
                    name=name,
                    type=type_value
                )

                # Save to the database
                await sync_to_async(position_side.save)()



In [6]:
async def add_markets():
    for status in MarketValues:
        name = status.value

        # Check if already exists in the database
        existing = await sync_to_async(Market.objects.filter(name=name).exists)()
        
        if not existing:
            # Create if it doesn't exist
            market = Market(
                name=name,
            )

            # Save to the database
            await sync_to_async(market.save)()

            



In [7]:
async def add_margin_mode():
    for status in MarginModeValues:
        name = status.value

        # Check if already exists in the database
        existing = await sync_to_async(MarginMode.objects.filter(name=name).exists)()
        
        if not existing:
            # Create if it doesn't exist
            margin = MarginMode(
                name=name,
            )

            # Save to the database
            await sync_to_async(margin.save)()

            


In [8]:
async def add_position_size():
    for status in PositionSideValues:
        name = status.value

        # Check if already exists in the database
        existing = await sync_to_async(PositionSide.objects.filter(name=name).exists)()
        
        if not existing:
            # Create if it doesn't exist
            position = PositionSide(
                name=name,
            )

            # Save to the database
            await sync_to_async(position.save)()

In [9]:
async def add_symbols():
    order_data = exchange.fetch_markets()

    spot_market = await sync_to_async(Market.objects.get)(name=MarketValues.SPOT.value)
    future_market = await sync_to_async(Market.objects.get)(name=MarketValues.FUTURES.value)

    for symbol in order_data:
        if symbol["quote"] == "USDT":
            market = spot_market if symbol["spot"] else future_market

            existing = await sync_to_async(Symbol.objects.filter(name=symbol["symbol"], market=market).exists)()

            if not existing:
                newSymbol = {
                    "name": symbol["symbol"],
                    "min_trade_amount": symbol["limits"]["amount"]["min"],
                    "quote": symbol["quote"],
                    "base": symbol["base"],
                    "market": market,
                }

                newSymbol = Symbol(**newSymbol)
                await sync_to_async(newSymbol.save)()

In [10]:
await add_margin_mode()
await add_markets()
await add_post_statuses()
await add_position_size()
await add_symbols()


# add channel info here
channels = [
    {
        'name': "TEST_FEYZIAN",
        'id': _config["CHANNEL_TEST_FEYZIAN"],
    },
    {
        'name': "FEYZIAN",
        'id': _config["CHANNEL_FEYZ"],
    },
    {
        'name': "ALI_BEY",
        'id': _config["CHANNEL_ALI_BEY"],
    },
    {
        'name': "TEST_ALI_BEYRANVAND",
        'id': _config["CHANNEL_TEST_ALI_BEYRANVAND"],
    },

]

for item in channels:
    await add_new_channel(item['id'], item['name'], False)